# Exploring the Entity Binding Circuit with SAEs

## Setup

In [1]:
import os, sys
import sys
import torch as t
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import numpy as np
import einops
from jaxtyping import Int, Float
import functools
from tqdm import tqdm
from IPython.display import display
from transformer_lens.hook_points import HookPoint
from transformer_lens import (
    utils,
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)
# import circuitsvis as cv
import os
import itertools
import random
import circuitsvis as cv

t.set_grad_enabled(False)

device = t.device("cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


## Recreating The Paper results with Gemma 9B
- Load Model (Done)
- Test Model on Capitals Task

In [3]:
model = HookedTransformer.from_pretrained('google/gemma-2-2b-it')

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer


In [4]:
'''model = model.to(device)
gemmascope_sae_release = "gemma-scope-2b-pt-res-canonical"
gemmascope_sae_id = "layer_20/width_16k/canonical"
gemma_2_2b_sae = SAE.from_pretrained(gemmascope_sae_release, gemmascope_sae_id, device=str(device))[0]
latent_idx = 12082

display_dashboard(sae_release=gemmascope_sae_release, sae_id=gemmascope_sae_id, latent_idx=latent_idx)'''

'model = model.to(device)\ngemmascope_sae_release = "gemma-scope-2b-pt-res-canonical"\ngemmascope_sae_id = "layer_20/width_16k/canonical"\ngemma_2_2b_sae = SAE.from_pretrained(gemmascope_sae_release, gemmascope_sae_id, device=str(device))[0]\nlatent_idx = 12082\n\ndisplay_dashboard(sae_release=gemmascope_sae_release, sae_id=gemmascope_sae_id, latent_idx=latent_idx)'

In [5]:
country_list = ['Thailand', 'Japan', 'Brazil', 'Morocco', 'Sweden', 'Kenya', 'Argentina', 'Australia', 'Egypt', 'Canada', 'Vietnam', 'Portugal', 'India', 'Norway', 'Mexico', 'Malaysia', 'Greece', 'Finland', 'Indonesia', 'Turkey', 'Chile', 'Ireland', 'Bangladesh', 'Denmark', 'Peru', 'Iceland', 'Colombia', 'Singapore', 'Austria', 'Nigeria', 'Croatia', 'Taiwan', 'Switzerland', 'Ghana', 'Cambodia', 'Poland', 'Nepal', 'Uruguay', 'Tanzania', 'Belgium', 'Jordan', 'Hungary', 'Bhutan', 'Maldives', 'Venezuela', 'Laos', 'Romania', 'Somalia', 'Mongolia', 'Uzbekistan']
name_list = ['Emma', 'James', 'Luna', 'Kai', 'Zara', 'Leo', 'Maya', 'Finn', 'Nova', 'Atlas', 'Rose', 'Sage', 'Jack', 'Ruby', 'Owen', 'Grace', 'Dean', 'Hope', 'Blake', 'Dawn', 'Cole', 'Faith', 'Reed', 'Sky', 'Jade', 'Wolf', 'Rain', 'Quinn', 'Blaze', 'Pearl', 'Felix', 'Iris', 'Seth', 'Dove', 'Drake', 'Joy', 'Axel', 'Fern', 'Stone', 'Wren', 'Grant', 'Hazel', 'Brooks', 'Ash', 'Reid', 'Sage', 'Clark', 'Skye', 'Blair', 'Scout']
capital_list = ['Bangkok', 'Tokyo', 'Brasilia', 'Rabat', 'Stockholm', 'Nairobi', 'Buenos Aires', 'Canberra', 'Cairo', 'Ottawa', 'Hanoi', 'Lisbon', 'New Delhi', 'Oslo', 'Mexico City', 'Kuala Lumpur', 'Athens', 'Helsinki', 'Jakarta', 'Ankara', 'Santiago', 'Dublin', 'Dhaka', 'Copenhagen', 'Lima', 'Reykjavik', 'Bogota', 'Singapore', 'Vienna', 'Abuja', 'Zagreb', 'Taipei', 'Bern', 'Accra', 'Phnom Penh', 'Warsaw', 'Kathmandu', 'Montevideo', 'Dodoma', 'Brussels', 'Amman', 'Budapest', 'Thimphu', 'Male', 'Caracas', 'Vientiane', 'Bucharest', 'Mogadishu', 'Ulaanbaatar', 'Tashkent']
capital_list = [f" {capital}" for capital in capital_list]
# country_list = [f" {country}" for country in country_list]
# name_list = [f" {name}" for name in name_list]

def get_one_token_lists(lists):
    assert len(lists) > 0
    assert type(lists[0]) is list
    list_len = len(lists[0])
    assert all([len(l) == list_len for l in lists])
    one_token_indices = []
    for i in range(list_len):
        is_one_token = True
        for j in range(len(lists)):
            name = lists[j][i]
            toks = model.to_tokens(name, prepend_bos=False)[0]
            if len(toks) > 1:
                is_one_token = False
                break
        if is_one_token:
            one_token_indices += [i]
    new_lists = [[] for _ in range(len(lists))]
    for idx in one_token_indices:
        for j in range(len(lists)):
            new_lists[j] += [lists[j][idx]]
    return new_lists

name_list_one_token = get_one_token_lists([name_list])[0]
country_list_one_token, capital_list_one_token = get_one_token_lists([country_list, capital_list])
len(country_list_one_token), len(capital_list_one_token), len(name_list_one_token)

(37, 37, 49)

In [ ]:
def get_prompt_capital(qn_subject, E_0, E_1, A_0, A_1):
    return f"""Answer the question based on the context below. Keep the answer short.
Context: {E_0} lives in the capital city of {A_0}.
{E_1} lives in the capital city of {A_1}.
Question: Which city does {qn_subject} live in?
Answer: {qn_subject} lives in the city of"""

def get_prompts(n, entities, attributes, get_prompt_function, answer_list = None):
    # sample n pairs of country+capital and n pairs of name+name
    prompts = []
    correct_answers = []
    wrong_answers = []
    
    # Sample n countries and n names without replacement
    A_0_list = random.choices(attributes, k=n)
    E_0_list = random.coices(entities, k=n)
    A_1_list = random.choices(attributes, k=n)
    E_1_list = random.coices(entities, k=n)
    
    for i in range(n):
        # For each prompt, we need 2 different countries and 2 different names
        E_0 = E_0_list[i]
        A_0 = A_0_list[i]
        E_1 = E_1_list[i]
        A_1 = A_1_list[i]
        qn_subject = random.choice([E_0, E_1])
        if answer_list is not None:
            correct_answer = answer_list[attributes.index(A_0)] if qn_subject == E_0 else answer_list[attributes.index(A_1)]
            wrong_answer = answer_list[attributes.index(A_1)] if qn_subject == E_0 else answer_list[attributes.index(A_0)]
        else:
            correct_answer = A_0 if qn_subject == E_0 else A_1
            wrong_answer = A_1 if qn_subject == E_0 else A_0

        prompt = get_prompt_function(qn_subject, E_0, E_1, A_0, A_1)
        prompts.append(prompt)
        correct_answers.append(correct_answer)
        wrong_answers.append(wrong_answer)
    
    return prompts, correct_answers, wrong_answers

n = 100
prompts, correct_answers, wrong_answers = get_prompts(n, country_list_one_token, name_list_one_token, get_prompt_capital, capital_list_one_token)
input_toks = model.to_tokens(prompts)
label_toks = model.to_tokens(correct_answers, prepend_bos=False)
input_toks.shape, label_toks.shape

TypeError: The number of choices must be a keyword argument: k=100

In [176]:
toks = model.generate(input_toks, max_new_tokens=1, do_sample=False)
model_out = toks[:, -1]
correct = (model_out == label_toks[:, 0])
acc = correct.float().mean()
acc

  0%|          | 0/1 [00:00<?, ?it/s]

tensor(1., device='cuda:0')